# Goal

This assignment Subset of the Caltech-UCSD Birds-200-2011 bird dataset. 

The **objective** is to produce a model that gives the highest possible accuracy on a test dataset containing the same categories.

# Guidelines
You should first clone the Github repository https://github.com/willowsierra/recvis20_a3. 
### main.py
Contains code for training and evaluating your models. 
Once training is completed, it produces a file kaggle.csv that lists the IDs of the test set images, along with their predicted label. This file should be uploaded to the Kaggle webpage, which will then produce a test accuracy score.

### Model.py
Model architecture is specified in model.py. 

Currently, a simple baseline model is provided. You can use this model or create your own. You are free to implement any approach covered in class, or in the research literature. 

Of course, tricks that you devise yourself are also encouraged. The test and training data are provided on the Kaggle competition data download page.

# What to hand-in
You should write a **1-page, double-column report in CVPR’21** format briefly presenting your approach and obtained results. This report should be uploaded in pdf format to the class Moodle by the deadline. You should also submit a **zip file with your code to the class Moodle (excluding any trained models)**.

In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd gdrive/'My Drive'/TP/A3Kaggle

/content/gdrive/My Drive/TP/A3Kaggle


Here we import all the necessary libraries

In [ ]:
import glob 
import os

from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import activations
from keras.preprocessing.image import ImageDataGenerator 
from keras.regularizers import l2, l1, l1_l2
import matplotlib.pyplot as plt 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tqdm

# CNN 


##Initialization of parameters

Here we initialize all the parameters of the CNN learning code.

1. insize : it's the input size 
1. ai, the size of the output of each conv2d layer
1. d01, do2 : Drop out in the first and second neural layer --> avoid Overfitting
1. n1, n2 : Number of nodes per layers (right after DenseNet) n2 is used only if we use 2 layers
1. L1 and L2 : Regularizers used in order to avoid overfitting.

In [ ]:
#Constantes

insize=128

a1=256    #the more the better : 256 is the highest I went
a2=64
a3=32
a4=128    #hidden
a5=512    #hidden

do1=0.1
do2=0.2
do3=0.5

n1=117   #117
n2=197   #197
n3=20    #hidden

L2=0.0001
L1=0.0001

##Model definition

In [ ]:
model = Sequential()
model.add(Conv2D(a1, (3, 3), activation='relu', padding='same', name='conv_1', 
                 input_shape=(insize, insize, 3)))
model.add(MaxPooling2D((2, 2), name='maxpool_1'))
model.add(Conv2D(a2, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(MaxPooling2D((2, 2), name='maxpool_2'))
model.add(Conv2D(a3, (3, 3), activation='relu', padding='same', name='conv_3'))
model.add(MaxPooling2D((2, 2), name='maxpool_3'))
model.add(Conv2D(a4, (3, 3), activation='relu', padding='same', name='conv_4'))
model.add(MaxPooling2D((2, 2), name='maxpool_4'))
#model.add(Conv2D(a5, (3, 3), activation='relu', padding='same', name='conv_5'))
#model.add(MaxPooling2D((2, 2), name='maxpool_5'))
model.add(Flatten())
model.add(Dense(n1, activation='relu', name='dense_1'))  #kernel_regularizer=l2(L1),
model.add(Dropout(do1))
model.add(Dense(n2, activation='relu', name='dense_2'))
model.add(Dropout(do2))
#model.add(Dense(n3,kernel_regularizer=l2(L1), activation='relu', name='dense_3'))
#model.add(Dropout(do3))
model.add(Dense(20, activation='softmax', name='output'))
 
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['accuracy'])

##Read dataset

In [ ]:
base_dir = 'bird_dataset'
train_dir = os.path.join(base_dir, 'train_images')
validation_dir = os.path.join(base_dir, 'val_images')
test_dir = os.path.join(base_dir, 'test_images')
 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
    )    #(rescale=1./255).  Ici on pourra s'amuser a faire de la data augmentation

test_datagen = ImageDataGenerator(
        rescale=1./255,
)   #(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=100,
        class_mode="categorical",
        shuffle=True,
        seed=42
)
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=100,
        class_mode="categorical",
        shuffle=True,
        seed=42
)

## Training

In [ ]:
history = model.fit(train_generator, 
                    steps_per_epoch=2, 
                    epochs=600, 
                    validation_data=validation_generator, 
                    validation_steps=1,
                    verbose=1)

### Saving the model

In [ ]:
model.save("models/181210")

## Predictions

In [ ]:
def read_image(x,y, shape=(insize,insize)):
    im_file = tf.io.read_file(x)
    im = tf.io.decode_jpeg(im_file, channels=3)
    im = tf.image.convert_image_dtype(im, tf.float32)
    im = tf.image.resize(im, shape)
    return im,y 

path = os.path.join(base_dir, 'test_images/mistery_category')
files = glob.glob(os.path.join(path,'*.jpg'))
preds = []
for f in tqdm.tqdm(files):
  im = read_image(f, None)[0]
  preds.append(np.argmax(model.predict(tf.expand_dims(im,axis=0))[0]))

## Writing predictions in CSV format 

In [ ]:
with open('181140.csv','w') as output:
  output.write("ID,Category\n")
  for f,pred in zip(files,preds):
    f=f.split('/')[-1]
    output.write("%s,%d\n"%(f[:-4], pred))

# Transfer Learning with DenseNet

## Initialization of parameters

Here we initialize all the parameters of the transfer learning code.

1.   insize : it's the input size (224 is best for DenseNet)
2.   d01 : Drop out in the first neural layer (right after DenseNet) --> avoid Overfitting
3.   n1, n2 : Number of nodes per layers (right after DenseNet) n2 is used only if we use 2 layers
4.   L1 and L2 : Regularizers used in order to avoid overfitting.

In [ ]:
insize=224

do1=0.3

n1=4096   #255 --> 0.6 0.8
n2=1024

L1=0.00001
L2=0.00001

##Read Dataset

###Read the dataset without performing no transformation on the images.

In [ ]:
#Read initial dataset

dataset_path = "bird_dataset/" 
#Dans le read image ajouter des transformations.
#faire l'union des datasets avec plusieurs transformations
# 1. D'abord creer plusieurs read_image avec transformations. 
# 2. make_tf

def read_image(x,y, shape=(insize, insize)):
    im_file = tf.io.read_file(x)
    im = tf.io.decode_jpeg(im_file, channels=3)
    im = tf.image.convert_image_dtype(im, tf.float32)
    im = tf.image.resize(im, shape)
    #im = data_augmentation(im, y)
    return im,y 


def make_tf_dataset(train_test="train", batch_size=32):
    path = f'{dataset_path}{train_test}_images'
    categories = sorted(os.listdir(path))
    print(f"Found {len(categories)} {train_test} classes.")
    x,y = [], []
    for i,cat in enumerate(categories):
        files = glob.glob(os.path.join(path,cat,'*.jpg')) ## Glob.glob ressort tous les files dans le dossier. path + categorie + file.jpg
        x.extend(files)
        y.extend(len(files)*[i])
    dataset = tf.data.Dataset.from_tensor_slices(x)
    label_dataset = tf.data.Dataset.from_tensor_slices(y)
    dataset = tf.data.Dataset.zip((dataset,label_dataset)) 
    dataset = dataset.shuffle(len(x))
    dataset = dataset.map(read_image)
    dataset = dataset.batch(batch_size)
    return dataset

### Read augmented dataset

In [ ]:
base_dir = 'bird_dataset'
train_dir = os.path.join(base_dir, 'train_images')
validation_dir = os.path.join(base_dir, 'val_images')
test_dir = os.path.join(base_dir, 'test_images')
unsup_dir = os.path.join(base_dir, 'unsupervised_images')

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
    )    #  Ici on pourra s'amuser a faire de la data augmentation


test_datagen = ImageDataGenerator(
        rescale=1./255,
)   #(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=64,
        class_mode="sparse",
        shuffle=True,
        seed=42
)
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=64,
        class_mode="sparse",
        shuffle=True,
        seed=42
)

##Model Definition

In [ ]:
pretrained_model = tf.keras.applications.DenseNet169(include_top=False,weights="imagenet", pooling = 'max') #VGG pourris, RESNET pourris, INCEPPTIONV3, NASNetLarge
pretrained_model.trainable = False
model = keras.Sequential([
    keras.layers.Input(shape=(insize, insize,3)),
    pretrained_model,
    keras.layers.Flatten(),
    keras.layers.Dense(n1,
                       activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #,activity_regularizer=regularizers.l2(1e-5)
                       ),
    keras.layers.Dropout(do1),
    keras.layers.Dense(n2, activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #activity_regularizer=regularizers.l2(1e-5)
                       ),
    keras.layers.Dropout(do2),
    keras.layers.Dense(20,activation="softmax")
])

model.compile("adam", "sparse_categorical_crossentropy", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)])

### Training

In [ ]:
model.fit(train_generator, 
                    epochs=20, 
                    validation_data=validation_generator,
                    validation_steps=1,
                    verbose=1)

### Predictions on test set

In [ ]:
def read_image(x,y, shape=(insize,insize)):
    im_file = tf.io.read_file(x)
    im = tf.io.decode_jpeg(im_file, channels=3)
    im = tf.image.convert_image_dtype(im, tf.float32)
    im = tf.image.resize(im, shape)
    return im,y 

path = os.path.join(base_dir, 'test_images/mistery_category')
files = glob.glob(os.path.join(path,'*.jpg'))
preds = []
for f in tqdm.tqdm(files):
  im = read_image(f, None)[0]
  preds.append(np.argmax(model.predict(tf.expand_dims(im,axis=0))[0]))

##Writing predictions in CSV format

In [ ]:
with open('DenseNet74.csv','w') as output:
  output.write("ID,Category\n")
  for f,pred in zip(files,preds):
    f=f.split('/')[-1]
    output.write("%s,%d\n"%(f[:-4], pred))

# Transfer learning with NA Birds

##Initialization of parameters

Here we initialize all the parameters of the transfer learning code.

1.   insize : it's the input size (224 is best for DenseNet)
2.   d01 : Drop out in the first neural layer (right after DenseNet) --> avoid Overfitting
3.   n1, n2 : Number of nodes per layers (right after DenseNet) n2 is used only if we use 2 layers
4.   L1 and L2 : Regularizers used in order to avoid overfitting.



In [ ]:
insize=224

do1=0.3

n1=4096   #255 --> 0.6 0.8
n2=1024

L1=0.00001
L2=0.00001

## Read Dataset


###Read the dataset without performing no transformation on the images.

In [ ]:
#Read initial dataset

dataset_path = "bird_dataset/" 
#Dans le read image ajouter des transformations.
#faire l'union des datasets avec plusieurs transformations
# 1. D'abord creer plusieurs read_image avec transformations. 
# 2. make_tf

def read_image(x,y, shape=(insize, insize)):
    im_file = tf.io.read_file(x)
    im = tf.io.decode_jpeg(im_file, channels=3)
    im = tf.image.convert_image_dtype(im, tf.float32)
    im = tf.image.resize(im, shape)
    #im = data_augmentation(im, y)
    return im,y 


def make_tf_dataset(train_test="train", batch_size=32):
    path = f'{dataset_path}{train_test}_images'
    categories = sorted(os.listdir(path))
    print(f"Found {len(categories)} {train_test} classes.")
    x,y = [], []
    for i,cat in enumerate(categories):
        files = glob.glob(os.path.join(path,cat,'*.jpg')) ## Glob.glob ressort tous les files dans le dossier. path + categorie + file.jpg
        x.extend(files)
        y.extend(len(files)*[i])
    dataset = tf.data.Dataset.from_tensor_slices(x)
    label_dataset = tf.data.Dataset.from_tensor_slices(y)
    dataset = tf.data.Dataset.zip((dataset,label_dataset)) 
    dataset = dataset.shuffle(len(x))
    dataset = dataset.map(read_image)
    dataset = dataset.batch(batch_size)
    return dataset

###Read augmented dataset

We have a quite small number of image per class. 

We therefore have a lot of chance to overfit the data.
Here we us augmentation in order to train our model on more data.

We use:

1.   Rescaling
1.   Rotation
1.   Width shift
1.   Height Shift
1.   Zoom
1.   horizontal flip

We use a quite high number on rotation shift and small numbers on the other parameters. 

In fact when we look at the dataset, we see that most pictures are often centered on the bird, with little space around. Therefore a high value for zoom or width and height wouldn't give a very good result.

We chose 40 for rotation shift because the bird can be taken in many different position. However we didn't see many birds upside down that's why we chose to put False for the vertical flip parameter.

We also introduce Unsup which is a part of the dataset of NAbirds that we user to train our algorithm later.

In [ ]:
base_dir = 'bird_dataset'
train_dir = os.path.join(base_dir, 'train_images')
validation_dir = os.path.join(base_dir, 'val_images')
test_dir = os.path.join(base_dir, 'test_images')
unsup_dir = os.path.join(base_dir, 'unsupervised_images')

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
    )    #  Ici on pourra s'amuser a faire de la data augmentation


test_datagen = ImageDataGenerator(
        rescale=1./255,
)   #(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=64,
        class_mode="sparse",
        shuffle=True,
        seed=42
)
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=64,
        class_mode="sparse",
        shuffle=True,
        seed=42
)

unsup_generator = test_datagen.flow_from_directory(
        unsup_dir,
        target_size=(insize, insize),
        color_mode="rgb",
        batch_size=1000,
        class_mode="sparse",
        shuffle=True,
        seed=42,
)

In [ ]:
train_dataset = make_tf_dataset()
val_dataset = make_tf_dataset(train_test="val")
unsup_dataset = make_tf_dataset(train_test="unsupervised")

##Model

Pretrained on DenseNet169 with ImageNet.

We simply add another layer of nodes.

And a last layr of classification for our 200 classes.

In [ ]:
pretrained_model = tf.keras.applications.DenseNet169(include_top=False,weights="imagenet", pooling = 'max') #VGG pourris, RESNET pourris, INCEPPTIONV3, NASNetLarge
pretrained_model.trainable = False
model = keras.Sequential([
    keras.layers.Input(shape=(insize, insize,3)),
    pretrained_model,
    keras.layers.Flatten(),
    keras.layers.Dense(n1,
                       activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #,activity_regularizer=regularizers.l2(1e-5)
                       ),
    keras.layers.Dropout(d01),
    keras.layers.Dense(n2, activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #activity_regularizer=regularizers.l2(1e-5)
                       ),
    keras.layers.Dropout(do2),
    keras.layers.Dense(300,activation="softmax")
])

model.compile("adam", "sparse_categorical_crossentropy", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)])

### Training on Nabirds

In [ ]:
model.fit(unsup_dataset, epochs=10, 
          #validation_data=val_dataset
          )

We save the model to re use it later.

In [ ]:
model.save("models/nabirdstrained200")
model.save("models/nabirdstrained200inh5.h5")

##Reconstructed model (just adding the dense 20 classification layer)

In [ ]:
reconstructed_model = keras.models.load_model("models/nabirdstrained200")

In [ ]:
reconstructed_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 1664)              12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1664)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              6819840   
_________________________________________________________________
dense_3 (Dense)              (None, 200)               819400    
Total params: 20,282,120
Trainable params: 20,123,720
Non-trainable params: 158,400
_________________________________________________________________


In [ ]:
reconstructed_model.pop()
reconstructed_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 1664)              12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1664)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              6819840   
Total params: 19,462,720
Trainable params: 19,304,320
Non-trainable params: 158,400
_________________________________________________________________


In [ ]:
reconstructed_model.trainable=False

In [ ]:
reconstructed_model.add(layers.Dense(20,activation="softmax"))

### Training of the reconstructed model

In [ ]:
reconstructed_model.fit(train_dataset, epochs=10, validation_data=val_dataset)  #reconstructed_model.fit(train_generator, 
                    #epochs=20, 
                    #validation_data=validation_generator,
                    #validation_steps=1,
                    #verbose=1)

##Reconstructed model adding a layer of n1 nodes

We remove the last 200 layer and we add one layer of n1 nodes, and  adding the dense 20 classification layer

In [ ]:
reconstructed_model2 = keras.models.load_model("models/nabirdstrained200")
reconstructed_model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 1664)              12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1664)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              6819840   
_________________________________________________________________
dense_3 (Dense)              (None, 200)               819400    
Total params: 20,282,120
Trainable params: 20,123,720
Non-trainable params: 158,400
_________________________________________________________________


In [ ]:
reconstructed_model2.trainable=False

In [ ]:
reconstructed_model2.pop()
reconstructed_model2.summary()

In [ ]:
reconstructed_model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 1664)              12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 1664)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              6819840   
Total params: 19,462,720
Trainable params: 0
Non-trainable params: 19,462,720
_________________________________________________________________


Definition 

In [ ]:
model2 = keras.Sequential([
    keras.layers.Input(shape=(insize, insize,3)),
    reconstructed_model2,
    #keras.layers.Dense(200,
     #                  activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #,activity_regularizer=regularizers.l2(1e-5)
    #                   ),
    #keras.layers.Dropout(do1),
    keras.layers.Dense(n1, activation=activations.relu
                       #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                       #bias_regularizer=regularizers.l2(1e-5),
                       #activity_regularizer=regularizers.l2(1e-5)
                       ),
    #keras.layers.Dropout(do1),
    keras.layers.Dense(20,activation="softmax")
])

model2.compile("adam", "sparse_categorical_crossentropy", metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)])

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 4096)              19462720  
_________________________________________________________________
dense_6 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_7 (Dense)              (None, 20)                81940     
Total params: 36,325,972
Trainable params: 16,863,252
Non-trainable params: 19,462,720
_________________________________________________________________


Without data augmentation

In [ ]:
model2.fit(train_dataset, epochs=100, validation_data=val_dataset)   #reconstructed_model.fit2(train_generator, 
                    #epochs=20, 
                    #validation_data=validation_generator,
                    #validation_steps=1,
                    #verbose=1)

##Predictions

In [ ]:
path = f'{dataset_path}test_images/mistery_category'
files = glob.glob(os.path.join(path,'*.jpg'))
preds = []
for f in tqdm.tqdm(files):
    im = read_image(f, None)[0]
    preds.append(np.argmax(model.predict(tf.expand_dims(im, axis=0))[0]))

100%|██████████| 517/517 [00:31<00:00, 16.21it/s]


In [ ]:
with open('NaBirdsW20056.csv', 'w') as output:
    output.write("Id,Category\n")
    for f, pred in zip(files, preds):
        f = f.split('/')[-1]
        output.write("%s,%d\n" % (f[:-4], pred))

In [ ]:
with open("CNN2layerstransferAugmented221530.csv") as f:
    print(f.read())

Id,Category
f3ffaa63cce4b75f414d93f65bd45743,13
f7a452821f33c8cd9152fb659ad68213,12
fe95bce0791a7015500d4b9f1d3d32c9,1
ffe6dc708419b819ea897d666e986ec6,8
f8a950b59250dbb8b189a07ad70d85ea,7
d716f3a09d7a8efa0ccac70d81b3b503,6
e7833a463fb9af679227107a3c36682c,0
baa237f0968b7c0b62f951979e6e4053,8
daff1aac7d1ea417939db6e7fc56428e,3
bd278c914f53755282f1d05873c180f3,1
cde3554433d0e297c17c74c9b243f47c,2
fa3db38c68c70898714a17ca83774158,7
fd5f28cd072603cb6e03c6e011808c0e,5
fb3dac8dd57973515860e1ee184f5f0a,1
ce85bf0371009d7b98cbd0828726b35a,9
f647602f147dce978d4a01019f88e4d2,7
e0d5d8f7658e9e7d9cfb33f434a1a36c,7
d08b43687503170c36402386df6a490f,11
f5a41d1b7638af9e2f3e1db1622364fd,6
b193593ec3ccc440508145cae0ef086e,13
d18c1c9b2849e5f77c2bc250635f0d95,9
f81b5d23407ed54fa3a99ed29fcda1a0,6
c05f7d929cc67ef267e0ffda404ad0f5,3
fef53a1dada4a77de35c609180d41936,2
c530c6986170aa1aefe7a930a56ad0db,7
cc17c00e53ba71456ca41f2ce568e87e,13
d50158395fdfdedcdd59ab6db5201e99,0
f5b06a47e3ec141ecc6590a543274f68,4
f43